In [53]:
import numpy as np
import imageio
import cv2

In [58]:
total_frames=500
def getFrames(vidfile):
    vid = cv2.VideoCapture(vidfile)
    frames = []
    endframe = True
    t = 0
    while endframe:
        if t>total_frames:
            break
        endframe,frame = vid.read()
        
        #resize images 

        scale_percent = 35 # percent of original size
        width = int(frame.shape[1] * scale_percent / 100)
        height = int(frame.shape[0] * scale_percent / 100)
        dim = (width, height)

      # resize image
        frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
        #print(frame)
        if endframe:
            frames.append(frame)
            #print("processing frame: ",t)
            t+=1
    return frames

In [76]:
def compute_dense_optical_flow(prev_image, current_image):
    old_shape = current_image.shape
    prev_image_gray = cv2.cvtColor(prev_image, cv2.COLOR_BGR2GRAY)
    current_image_gray = cv2.cvtColor(current_image, cv2.COLOR_BGR2GRAY)

    assert current_image.shape == old_shape

    flow = cv2.calcOpticalFlowFarneback(prev_image_gray,current_image_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    hsv = np.zeros_like(prev_image)
    hsv[...,1] = 255

    
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
#     rgb = cv2.cvtColor(hsv,cv2.COLOR_BGR2GRAY)
    
    

    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR) 
    return rgb

In [85]:
images_data= getFrames('tests/tdc1.mov')

In [86]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER,
         10, 1.0)

flags = cv2.KMEANS_RANDOM_CENTERS

with imageio.get_writer('Optflow2.gif', mode='I',fps=65) as gifwriter:
    for i in range(0,len(images_data)-1):#trying to get better optical flow by comparing distant frames,                                                                 #instead of consecutive 

        image = compute_dense_optical_flow(images_data[i],images_data[i+1])
        #image = (image/256).astype('uint8')

        gifwriter.append_data(image)



In [87]:
!gifsicle -i Optflow2.gif -O3 --colors 16 -o Optflow-opt2.gif